<a href="https://colab.research.google.com/github/NatalyGri/DSL/blob/main/Job_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Контекстно-свободная грамматика представляется словарем:
```
{'toks': set(token), 'vars': dict(var: definition), 'hvar': var}
token : (class, value)
class : int
value : str
var : str                 # имя нетерминала
definition : list(rule)
rule : list(var | token)  # правая часть правила
```
# 1) Удаление сторонних нетерминалов.

**Сторонние нетерминальные символы** - непродуктивные и недостежимые символы.

**Нетерминал А** называется **достижимым** если он выводим из главного нетерминального символа.

**Нетерминал А** называется **продуктивным** если из него можно вывести цепочку терминальных символов.

Чтобы очистить грамматику, мы находим все продуктивные и достижимые нетерминалы, а затем убираем все ненужное.

1) Находим продуктивные нетерминалы, просматривая каждый нетерминал и его продуционные правила, отмечая те нетерминалы, правая часть правила которых содержит только токены или другие продуктивные нетерминалы.

2) Находим достижимые нетерминалы. Первый достижимый нетерминал - это главный нетерминал, затем мы добавляем все нетерминалы, содержащиеся в его 
продуционных правилах, в множество достижимых нетерминалов и повторяем эту операцию с обновленным множеством достижимых нетерминалов, пока оно увеличиваеться.

3) Сравниваем исходное множество нетерминалов с объединенным продуктивных и достижимых нетерминалов и включаем только продуктивные нетерминалы с правилами, содержащими только достижимые нетерминалы. 

In [5]:
def clean_external(grammar):

  terminal = grammar['toks']
  nterminal = grammar['vars']
  non_extrnal = set()
  
  #вспомогательная функция для определения продуктивности нетерминала
  def is_non_external(definition):
    return any(map(check_rule, definition))

  #вспомогательная функция для анализа правой части правила на непродуктивные нетерминалы
  def check_rule(rule):
    return all(map((lambda part: part in terminal.union(non_extrnal)), rule))

  #вспомогательная функция для "очищения" правил от непродуктивныъ нетерминалов
  def clean_definition(definition):
    new_definition = []
    for rule in definition:
        if check_rule(rule):
          new_definition.append(rule)
    return new_definition


  #ищем продуктивные нетерминалы
  new_non_extrnal= True 
  while new_non_extrnal:
    new_non_extrnal = False
    for nterm, definition in nterminal.items():
      if nterm not in non_extrnal and is_non_external(definition):
        non_extrnal.add(nterm)
        new_non_extrnal = True


  #ищем достижимые нетерминалы
  finded = set(grammar['hvar'])
  find_new_finded = True
  while find_new_finded:
    find_new_finded = False
    for nterm, definition in nterminal.items():
      if nterm in finded:
        for rule in definition:
          new_finded = finded.union(set(filter(lambda part: part in nterminal, rule)))
          find_new_finded = not (len(finded) == len(finded))
          finded = new_finded

  #получаем множество сторонних нетерминалов
  non_extrnal = non_extrnal.union(finded)
  
  #удаляем из грамматики все посторонних нетерминалы и правила в которых они содержатся)
  new_nterminal = dict()
  for nterm, definition in nterminal.items():
    if nterm in non_extrnal:
      new_definition = clean_definition(definition)
      if nterm not in new_nterminal.keys() and new_definition:
        new_nterminal[nterm] = []
        new_nterminal[nterm] = new_definition
      
  grammar['vars'] = new_nterminal

  return grammar

# 2) Определение исчезающих символов
Чтобы найти все исчезающие нетерминалы, мы рассматриваем каждый нетерминал и его продуционные правила. Если мы находим правило вида α->ε или α->β, когда β->>ε, мы добавляем этот нетерминал в множество исчезающих и повторяем все действия, пока множество исчезающих нетерминалов увеличиваеться.

In [7]:
def vanishing(grammar, empt):
  teminals = grammar['toks']
  nteminals = grammar['vars']
  vanishing_set = set() 
  
  #вспомогательная функция для определения исчезаемости нетерминала
  def is_vanishing(definition):
    for rule in definition: 
       if all(map((lambda symbol: symbol == empt or symbol in vanishing_set), rule)): return True
    return False

  #ищем исчезающие нетерминалы
  new_vanishing_flag = True 
  while new_vanishing_flag:
    new_vanishing_flag = False
    for nterm, definition in nteminals.items():
      if nterm not in vanishing_set and is_vanishing(definition) :
        vanishing_set.add(nterm)
        new_vanishing_flag = True
          
  return vanishing_set.difference(grammar['hvar']) #исключаем главный нетерминал

In [8]:
grammar = {
    'toks': set( [
        (0, 'e'), 
        (1, 'a'), 
        (2, 'b'), 
        (3, 'c'),
        (4, 'd'),
        (5, 'f'),
    ] ),

    'vars': {
        'A' : [['A', (1, 'a')], 
               ['B', 'F'],
               ['C', (3, 'c'), 'D']],
        'B' : [['D', (1, 'c')]],
        'C' : [['B'],
               [(0, 'e')]],
        'D' : [['B', (4, 'd')],
               ['C', (2, 'b')]],
        'F' : [[(0, 'e')],
               [(2, 'b'), 'B'],
               ['C']],
        'S' : [['A']]
    },
    'hvar': 'S'
}

print("Grammar without unusefull(external) non-terminals: \n", clean_external(grammar))
print("Vanishing: ", vanishing(grammar, (0, 'e')))

Grammar without unusefull(external) non-terminals: 
 {'toks': {(4, 'd'), (3, 'c'), (1, 'a'), (5, 'f'), (2, 'b'), (0, 'e')}, 'vars': {'A': [['A', (1, 'a')], ['C', (3, 'c'), 'D']], 'C': [[(0, 'e')]], 'D': [['C', (2, 'b')]], 'F': [[(0, 'e')], ['C']], 'S': [['A']]}, 'hvar': 'S'}
Vanishing:  {'F', 'C'}
